In [1]:
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.nn import functional as F

from matplotlib import pyplot as plt
from IPython.display import clear_output

In [2]:
class Classifier(nn.Module):
    def __init__(self, output_bias=None):
        super(Classifier, self).__init__()
        self.dense1 = nn.Linear(7, 20)
        self.dropout = nn.Dropout(0.5)
        self.output = nn.Linear(20, 2)

        if output_bias is not None:
            self.output.bias.data.fill_(output_bias)

        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.dense1(x))
        x = self.dropout(x)
        x = self.output(x)
        return x

model = Classifier()

In [9]:
from torch.utils.data import DataLoader, TensorDataset

data = pd.read_csv('data_part1.csv')

X = data[['dis', 'iou', 'conf', 'dx1', 'dy1', 'dx2', 'dy2']].values
y = data['problem'].values

X_tensor = torch.FloatTensor(X)
y_tensor = torch.LongTensor(y)

dataset = TensorDataset(X_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [10]:
neg, pos = np.bincount(data['problem'])
initial_bias = np.log([pos / neg])[0]

In [11]:
model = Classifier(output_bias=initial_bias)

In [13]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

class_counts = torch.bincount(y_tensor)
class_weights = torch.tensor([1.0, class_counts[0]/class_counts[1].float()])
print(class_weights)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def train_model(model, dataloader, criterion, optimizer, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        all_preds = []
        all_labels = []

        for images, labels in dataloader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

        # Вычисление метрик
        acc = accuracy_score(all_labels, all_preds)
        prec = precision_score(all_labels, all_preds, zero_division=0)
        rec = recall_score(all_labels, all_preds, zero_division=0)
        f1 = f1_score(all_labels, all_preds, zero_division=0)

        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'Loss: {total_loss/len(dataloader):.4f} | Accuracy: {acc:.4f}')
        print(f'Precision: {prec:.4f} | Recall: {rec:.4f} | F1: {f1:.4f}')
        print('-' * 50)

# Запуск обучения
train_model(model, dataloader, criterion, optimizer, num_epochs=20)

tensor([ 1.0000, 45.8736])
Epoch 1/20:
Loss: 0.3158 | Accuracy: 0.8575
Precision: 0.1181 | Recall: 0.8781 | F1: 0.2082
--------------------------------------------------
Epoch 2/20:
Loss: 0.3039 | Accuracy: 0.8566
Precision: 0.1179 | Recall: 0.8826 | F1: 0.2080
--------------------------------------------------
Epoch 3/20:
Loss: 0.3200 | Accuracy: 0.8487
Precision: 0.1126 | Recall: 0.8849 | F1: 0.1997
--------------------------------------------------
Epoch 4/20:
Loss: 0.3095 | Accuracy: 0.8450
Precision: 0.1121 | Recall: 0.9052 | F1: 0.1995
--------------------------------------------------
Epoch 5/20:
Loss: 0.3079 | Accuracy: 0.8507
Precision: 0.1143 | Recall: 0.8894 | F1: 0.2026
--------------------------------------------------
Epoch 6/20:
Loss: 0.2971 | Accuracy: 0.8558
Precision: 0.1191 | Recall: 0.9007 | F1: 0.2104
--------------------------------------------------
Epoch 7/20:
Loss: 0.2943 | Accuracy: 0.8483
Precision: 0.1152 | Recall: 0.9142 | F1: 0.2045
-----------------------

In [15]:
torch.save(model.state_dict(), 'model_weights.pth')